In [4]:
import pandas as pd
import datetime
import numpy as np
import boto3
import logging
from io import StringIO
from dotenv import load_dotenv
from pathlib import Path
import os
import polars as pl
from functions_mo import final_result, calculate_volume_distribution

In [5]:
env_path = Path('C:/Users/Nata_/Documents/Etapa_1_proyecto/Simulador_mano_de_obra/mano_de_obra_agrovid/var.env')
load_dotenv(dotenv_path=env_path)
AWS_PROFILE = os.getenv("aws_profile") 
SSM_PARAMETER_BUCKET_INPUT_FILES_NAME = os.getenv('SSM_PARAMETER_BUCKET_INPUT_FILES_NAME')
SSM_PARAMETER_BUCKET_OUTPUT_FILES_NAME = os.getenv('SSM_PARAMETER_BUCKET_OUTPUT_FILES_NAME')
REGION = os.getenv('REGION')


def get_ssm_parameters(aws_profile, region, parameter_names):
  
    session = boto3.Session(profile_name=aws_profile)
    ssm_client = session.client('ssm', region_name=region)

    parameters = {}
    for param_name in parameter_names:
        try:
            # Retrieve the parameter value
            response = ssm_client.get_parameter(Name=param_name, WithDecryption=True)
            parameters[param_name] = response['Parameter']['Value']
        except ssm_client.exceptions.ParameterNotFound:
            print(f"Parameter {param_name} not found.")
        except Exception as e:
            print(f"Error retrieving parameter {param_name}: {e}")

    return parameters

parameter_names = [
        SSM_PARAMETER_BUCKET_INPUT_FILES_NAME,
        SSM_PARAMETER_BUCKET_OUTPUT_FILES_NAME
    ]

ssm_parameters = get_ssm_parameters(AWS_PROFILE, REGION, parameter_names)

bucket_input_file = ssm_parameters.get(SSM_PARAMETER_BUCKET_INPUT_FILES_NAME)
bucket_output_file = ssm_parameters.get(SSM_PARAMETER_BUCKET_OUTPUT_FILES_NAME)



def download_csv_from_s3(aws_profile, bucket_name, file_keys):
    session = boto3.Session(profile_name=aws_profile)
    s3_client = session.client('s3')
    data_frames = {}

    for file_key in file_keys:
        obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)
        data = obj['Body'].read().decode('utf-8')
        sep = ',' if file_key.endswith('.csv') else ';'
        data_frames[file_key] = pd.read_csv(StringIO(data), sep=sep)
    
    return data_frames


file_keys = ['er_simulado/modulo_ingresos/2024-07-24/ingreso.csv', 
             'er_simulado/modulo_ingresos/2024-07-24/EXCEDENTE.csv', 
             'er_simulado/modulo_ingresos/2024-07-24/platano.csv']

data_frames = download_csv_from_s3(AWS_PROFILE, bucket_input_file, file_keys)

# Acceder a los DataFrames descargados
surplus = data_frames['er_simulado/modulo_ingresos/2024-07-24/EXCEDENTE.csv']
revenue_file = data_frames['er_simulado/modulo_ingresos/2024-07-24/ingreso.csv']



2025-01-22 19:18:21,266 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-01-22 19:18:22,294 - INFO - Found credentials in shared credentials file: ~/.aws/credentials


## Importar los archivos

In [6]:
box_volum = pd.read_csv("volumen.csv", sep=';')
box_volum.head()

,Unnamed: 0,CONCEPTO,FINCA,SEMANA 1,SEMANA 2,SEMANA 3,SEMANA 4,SEMANA 5,SEMANA 6,SEMANA 7,SEMANA 8,SEMANA 9,SEMANA 10,SEMANA 11,SEMANA 12,SEMANA 13,SEMANA 14,SEMANA 15,SEMANA 16,SEMANA 17,SEMANA 18,SEMANA 19,SEMANA 20,SEMANA 21,SEMANA 22,SEMANA 23,SEMANA 24,SEMANA 25,SEMANA 26,SEMANA 27,SEMANA 28,SEMANA 29,SEMANA 30,SEMANA 31,SEMANA 32,SEMANA 33,SEMANA 34,SEMANA 35,SEMANA 36,SEMANA 37,SEMANA 38,SEMANA 39,SEMANA 40,SEMANA 41,SEMANA 42,SEMANA 43,SEMANA 44,SEMANA 45,SEMANA 46,SEMANA 47,SEMANA 48,SEMANA 49,SEMANA 50,SEMANA 51,SEMANA 52
0,0,CAJAS,ENANO,7969.0,8319.0,7813.0,7359.0,7989.0,5479.0,6037.0,6918.0,8895.0,8528.0,7881.0,10996.0,5932.0,9973.0,9081.0,9736.0,3075.0,6259.0,7684.0,7335.0,7933.0,9271.0,9187.0,9112.0,9794.0,9651.0,9473.0,9704.0,9755.0,9941.0,9967.0,9764.0,9638.0,9249.0,8932.0,8832.0,8854.0,8523.0,8166.0,8117.0,8542.0,8323.0,9164.0,9406.0,10244.0,9851.0,9751.0,9302.0,10343.0,10606.0,10349.0,9936.0
1,1,CAJAS,EVA,4098.0,6617.0,7238.0,7600.0,6414.0,4585.0,4992.0,5411.0,6998.0,5030.0,6915.0,8672.0,5511.0,8557.0,7658.0,6966.0,4637.0,4482.0,5003.0,5272.0,5683.0,5821.0,5436.0,5537.0,5980.0,5948.0,5860.0,6182.0,6379.0,6793.0,7191.0,7141.0,7266.0,6995.0,7244.0,7539.0,7895.0,7696.0,7549.0,7373.0,7799.0,7070.0,7407.0,7045.0,7287.0,6578.0,6586.0,6073.0,6218.0,6091.0,5903.0,5791.0
2,2,CAJAS,SAMI,4560.0,8771.0,9549.0,9418.0,8004.0,6011.0,5031.0,6875.0,8082.0,9268.0,10144.0,11023.0,6777.0,8463.0,9551.0,8376.0,4891.0,5626.0,6202.0,6690.0,6935.0,6782.0,6441.0,6791.0,7802.0,9330.0,10072.0,10491.0,10821.0,11369.0,11088.0,10469.0,10392.0,10567.0,10333.0,10346.0,10179.0,10164.0,9752.0,9622.0,9392.0,8219.0,8233.0,8250.0,8530.0,7911.0,7875.0,7546.0,8216.0,7893.0,7937.0,7645.0
3,3,CAJAS,VEGA,3625.0,3448.0,4037.0,2996.0,3455.0,2949.0,3097.0,3034.0,3528.0,4040.0,5456.0,5022.0,3034.0,4532.0,5040.0,4434.0,2609.0,3194.0,3464.0,3423.0,3382.0,3423.0,3399.0,3406.0,3547.0,3563.0,3586.0,3568.0,3713.0,3801.0,3945.0,3897.0,4036.0,4267.0,4484.0,4613.0,4705.0,4680.0,4758.0,4699.0,4928.0,4537.0,4869.0,4584.0,4547.0,3897.0,3637.0,3361.0,3460.0,3426.0,3303.0,3147.0
4,4,CAJAS,FEDERICA,5265.0,5243.0,3467.0,3098.0,3646.0,4033.0,4654.0,4113.0,5740.0,5822.0,5380.0,7296.0,3453.0,5413.0,6927.0,7337.0,1839.0,4916.0,6789.0,6781.0,8183.0,8249.0,8102.0,7695.0,7826.0,8369.0,8470.0,8596.0,8760.0,8980.0,9444.0,9611.0,9242.0,8671.0,8985.0,9627.0,9700.0,9431.0,9598.0,9472.0,10082.0,9743.0,10303.0,9370.0,9802.0,8779.0,8725.0,8484.0,9021.0,8505.0,7837.0,7814.0


In [7]:
cut = pd.read_csv('corte_y_empaque.csv', sep = ';')
cut.head()

,ID,FINCA,GRUPO,LABOR,MAESTRA,TARIFA,REF,PROMEDIADO,PRESTACIONES,FACTOR
0,100001,ENANO,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
1,100001,EVA,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
2,100001,SAMI,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
3,100001,VEGA,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
4,100001,FEDERICA,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0


In [8]:
volum_distribution = pd.read_csv('distribucion_volumen.csv', sep = ';')
volum_distribution.head()

,SEMANA,SEM 1,SEM 2,SEM 3,SEM 4,SEM 5,SEM 6,SEM 7,SEM 8,SEM 9,SEM 10,SEM 11,SEM 12,SEM 13,SEM 14,SEM 15,SEM 16,SEM 17,SEM 18,SEM 19,SEM 20,SEM 21,SEM 22,SEM 23,SEM 24,SEM 25,SEM 26,SEM 27,SEM 28,SEM 29,SEM 30,SEM 31,SEM 32,SEM 33,SEM 34,SEM 35,SEM 36,SEM 37,SEM 38,SEM 39,SEM 40,SEM 41,SEM 42,SEM 43,SEM 44,SEM 45,SEM 46,SEM 47,SEM 48,SEM 49,SEM 50,SEM 51,SEM 52
0,ENERO,1,1,1,1,0.6,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0
1,FEBRERO,0,0,0,0,0.4,1,1,1,0.8,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0
2,MARZO,0,0,0,0,0.0,0,0,0,0.2,1,1,1,1,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0
3,ABRIL,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,1,1,1,1,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0
4,MAYO,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.6,1,1,1,1,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0


## Corte y empaque

In [9]:
volum_distribution_transformed = final_result(volum_distribution, 2024, 1)
box_volum_transformed = final_result(box_volum, 2024, 3)

2025-01-22 19:18:23,489 - INFO - Fechas calculadas para cada semana: [datetime.datetime(2024, 1, 6, 0, 0), datetime.datetime(2024, 1, 13, 0, 0), datetime.datetime(2024, 1, 20, 0, 0), datetime.datetime(2024, 1, 27, 0, 0), datetime.datetime(2024, 2, 3, 0, 0), datetime.datetime(2024, 2, 10, 0, 0), datetime.datetime(2024, 2, 17, 0, 0), datetime.datetime(2024, 2, 24, 0, 0), datetime.datetime(2024, 3, 2, 0, 0), datetime.datetime(2024, 3, 9, 0, 0), datetime.datetime(2024, 3, 16, 0, 0), datetime.datetime(2024, 3, 23, 0, 0), datetime.datetime(2024, 3, 30, 0, 0), datetime.datetime(2024, 4, 6, 0, 0), datetime.datetime(2024, 4, 13, 0, 0), datetime.datetime(2024, 4, 20, 0, 0), datetime.datetime(2024, 4, 27, 0, 0), datetime.datetime(2024, 5, 4, 0, 0), datetime.datetime(2024, 5, 11, 0, 0), datetime.datetime(2024, 5, 18, 0, 0), datetime.datetime(2024, 5, 25, 0, 0), datetime.datetime(2024, 6, 1, 0, 0), datetime.datetime(2024, 6, 8, 0, 0), datetime.datetime(2024, 6, 15, 0, 0), datetime.datetime(2024, 6,

In [20]:
volum_distribution_matrix = volum_distribution_transformed.iloc[:, 1:]
box_id_100001 = box_volum_transformed[box_volum_transformed['CONCEPTO'] == 'CAJAS'].iloc[:, 3:]
box_farm = box_volum_transformed[box_volum_transformed['CONCEPTO'] == 'CAJAS']
new_columns = ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08',
       '2024-09', '2024-10', '2024-11', '2024-12']

In [22]:
labor_cic = calculate_volume_distribution(box_id_100001, volum_distribution_matrix, box_farm, new_columns)

2025-01-22 19:25:15,921 - INFO - Iniciando el cálculo de la matriz de distribución de volumen.
2025-01-22 19:25:15,922 - INFO - Procesando el mes: 0
2025-01-22 19:25:15,927 - INFO - Procesando el mes: 1
2025-01-22 19:25:15,950 - INFO - Procesando el mes: 2
2025-01-22 19:25:15,963 - INFO - Procesando el mes: 3
2025-01-22 19:25:15,977 - INFO - Procesando el mes: 4
2025-01-22 19:25:15,977 - INFO - Procesando el mes: 5
2025-01-22 19:25:15,996 - INFO - Procesando el mes: 6
2025-01-22 19:25:15,996 - INFO - Procesando el mes: 7
2025-01-22 19:25:16,016 - INFO - Procesando el mes: 8
2025-01-22 19:25:16,028 - INFO - Procesando el mes: 9
2025-01-22 19:25:16,031 - INFO - Procesando el mes: 10
2025-01-22 19:25:16,046 - INFO - Procesando el mes: 11
2025-01-22 19:25:16,061 - INFO - Matriz de resultados generada.
2025-01-22 19:25:16,061 - INFO - Matriz de resultados transpuesta y columnas renombradas.
2025-01-22 19:25:16,061 - INFO - DataFrame final concatenado con los datos de volumen transformados.


In [27]:
labor_cic


,FINCA,CONCEPTO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,CAJAS,36253.4,28745.6,35116.0,34368.6,35978.4,37744.0,44853.2,41569.8,35998.4,40047.4,41029.2,41234.0
1,EVA,CAJAS,29401.4,23152.0,27527.6,29610.8,24468.2,22901.0,29528.6,31522.4,32153.6,33810.4,27933.0,24003.0
2,SAMI,CAJAS,37100.4,27584.2,38828.4,33531.4,29984.6,30364.0,49405.8,46196.2,42365.4,40141.6,33512.0,31691.0
3,VEGA,CAJAS,16179.0,13284.4,18257.6,17892.6,15608.4,13915.0,17035.0,18262.0,19695.8,21760.4,16358.8,13336.0
4,FEDERICA,CAJAS,19260.6,18850.4,23099.0,23482.4,32951.6,31992.0,40472.4,40286.6,40250.4,45201.6,37664.0,33177.0
5,NEERLANDIA,CAJAS,21160.8,15766.4,22825.8,29697.8,25612.2,22451.0,28206.4,30207.6,29904.2,34381.4,30621.4,29271.0
6,SAN ANTONIO,CAJAS,28258.2,21984.8,28497.0,28030.2,27632.8,25564.0,33204.6,33724.4,33004.6,36158.0,30585.4,26287.0
7,MRS,CAJAS,13403.8,8874.6,10766.6,7980.2,8827.8,8977.0,11642.6,11140.4,10971.6,12075.4,10577.0,9186.0
8,DON FUAD,CAJAS,31850.0,26279.8,38492.2,44479.2,32925.8,30264.0,38753.4,44285.6,46008.4,50712.2,39997.4,34170.0
9,EUFEMIA,CAJAS,54256.0,42183.8,54314.2,51532.0,59857.0,61195.0,69885.2,67362.8,66722.6,71515.2,63304.2,60802.0


In [40]:
def calcular_resultado(matriz_a: pd.DataFrame, matriz_b: pd.DataFrame) -> pd.DataFrame:

    # Convertir las columnas en formato datetime

    # Eliminar columnas no relacionadas (si existen)
    matriz_a_numeric = matriz_a
    matriz_b_numeric = matriz_b

    # Crear un DataFrame para almacenar los resultados
    resultados = pd.DataFrame()

    # Iterar por las filas de la matriz A (cada mes)
    for i, fila_a in matriz_a_numeric.iterrows():
        # Multiplicar cada fila de B por la fila actual de A
        producto = matriz_b_numeric.multiply(fila_a.values, axis=1)
        # Sumar los resultados para cada fila de B
        suma_producto = producto.sum(axis=1)
        # Agregar el resultado como una nueva columna en el DataFrame
        resultados[fila_a.name] = suma_producto

    # Renombrar las columnas con los meses


    return suma_producto

In [41]:
calcular_resultado(volum_distribution_matrix, box_id_100001)

0     41234.0
1     24003.0
2     31691.0
3     13336.0
4     33177.0
5     29271.0
6     26287.0
7      9186.0
8     34170.0
9     60802.0
10     9559.0
11    81285.0
12    37114.0
13    79172.0
14     9527.0
15    10153.0
16    50302.0
17    37539.0
18    32680.0
19        0.0
20    17529.0
21    12269.0
22        0.0
23        0.0
24    11545.0
25    18521.0
26     9126.0
27    14348.0
28     8290.0
29    35319.0
30    18620.0
31     8823.0
32    10033.0
33    48376.0
34    28063.0
35    39696.0
36    35658.0
37    23194.0
38     2361.0
39    12227.0
40    53028.0
41    26200.0
42    11616.0
43    15404.0
44     5070.0
45    43955.0
dtype: float64

In [18]:
cut

,ID,FINCA,GRUPO,LABOR,MAESTRA,TARIFA,REF,PROMEDIADO,PRESTACIONES,FACTOR
0,100001,ENANO,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
1,100001,EVA,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
2,100001,SAMI,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
3,100001,VEGA,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
4,100001,FEDERICA,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0
...,...,...,...,...,...,...,...,...,...,...
1053,100022,DON PEDRO,EMPAQUE,AJUSTE DE EMPAQUE,ORG NORTE,1.0,AJUSTE,G04,0.3887,0.0
1054,100022,LA VELA,EMPAQUE,AJUSTE DE EMPAQUE,ORG NORTE,1.0,AJUSTE,G04,0.3887,0.0
1055,100022,ESPERANZA,EMPAQUE,AJUSTE DE EMPAQUE,ORG NORTE,1.0,AJUSTE,G04,0.3887,0.0
1056,100022,LINA MARGARITA,EMPAQUE,AJUSTE DE EMPAQUE,PLAT,0.0,AJUSTE,G04,0.3887,0.0
